In [31]:
pip install gradio

In [ ]:
pip install fpdf

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import gradio as gr
from fpdf import FPDF
import os

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load and preprocess the data
data = pd.read_csv('dataset.csv')
description_data = pd.read_csv('symptom_Description.csv')
precaution_data = pd.read_csv('symptom_precaution.csv')
data.fillna('unknown', inplace=True)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
# Lowercase, strip, and replace underscores for consistency
for col in data.columns[1:]:
    data[col] = data[col].str.lower().str.strip().str.replace('_', ' ')

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(12, 12))
sns.countplot(y='Disease', data=data, order=data['Disease'].value_counts().index)
plt.title('Disease Distribution')
plt.show()

In [ ]:
#To understand the correlation between different symptoms and the disease.

encoded_data = data.copy()
label_encoder = LabelEncoder()
for col in encoded_data.columns:
    encoded_data[col] = label_encoder.fit_transform(encoded_data[col])

plt.figure(figsize=(16, 12))
sns.heatmap(encoded_data.corr(), annot=True, fmt='.2f', cmap='coolwarm', cbar=True)
plt.title('Correlation Matrix of Features')
plt.show()

In [ ]:
#Distribution of each symptom to see how they are spread across the dataset.
for col in data.columns[1:]:
    plt.figure(figsize=(10, 6))
    sns.histplot(data[col], kde=False, bins=30)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
# Identify symptom columns (assume 'Disease' is the first column)
symptom_columns = data.columns[1:]

# Gather all unique symptoms across all symptom columns
unique_symptoms = pd.Series(data[symptom_columns].values.ravel()).dropna().unique()
symptom_to_number = {symptom: num for num, symptom in enumerate(unique_symptoms, start=101)}

# Print all unique values
print("Unique values from the entire dataset:")
for value in unique_symptoms:
    print(value)

In [ ]:
# Create a dictionary to map each unique symptom to a unique number starting from 101
symptom_to_number = {symptom: num for num, symptom in enumerate(unique_symptoms, start=101)}

# Replace symptoms in the symptom columns with their corresponding unique numbers
for col in symptom_columns:
    data[col] = data[col].map(symptom_to_number).fillna(0).astype(int)

# Print the dictionary of unique symptoms and their corresponding numbers
print("Unique symptoms and their corresponding numbers:")
for symptom, number in symptom_to_number.items():
    print(f"{symptom}: {number}")

In [ ]:
# Encode the 'Disease' column
label_encoder = LabelEncoder()
data['Disease'] = label_encoder.fit_transform(data['Disease'])

In [ ]:
# Get the mapping of each distinct disease to its encoded value
disease_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Print all the distinct diseases with their encoded values
for disease, encoding in disease_mapping.items():
    print(f"{disease}: {encoding}")

In [ ]:
# Prepare training and test sets
X = data[data.columns[1:]]
y = data['Disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize Random Forest model
rf = RandomForestClassifier(random_state=42)

# GridSearchCV for parameter tuning
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters found
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Train the model with the best parameters
best_rf_model = RandomForestClassifier(**best_params, random_state=42)
best_rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_rf = best_rf_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_rf)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_rf)}")

# Confidence Score
def get_confidence_score(symptoms_encoded):
    proba = best_rf_model.predict_proba(symptoms_encoded)
    confidence_score = round(max(proba[0]) * 100, 2)
    return confidence_score

    # PDF export
def export_to_pdf(symptoms, prediction, confidence_score, description, precaution):
    pdf = FPDF(format='A4')  # Standard A4 size for more space
    pdf.add_page()

    # Set title font and size
    pdf.set_font("Arial", style='B', size=16)
    pdf.cell(200, 10, txt="Disease Prediction Report", ln=True, align='C')
    pdf.ln(10)  # Add a blank line for spacing

    # Increase margin for neatness
    pdf.set_left_margin(15)
    pdf.set_right_margin(15)

    # Change to regular font for content
    pdf.set_font("Arial", size=12)

    # Define a function to wrap long text in the PDF
    def multi_line_text(label, content):
        pdf.set_font("Arial", style='B', size=12)
        pdf.cell(0, 10, label, ln=True)
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, content)
        pdf.ln(2)  # Small space after each section

    # Adding sections with wrapping and spacing
    multi_line_text("Symptoms:", symptoms)
    multi_line_text("Predicted Disease:", prediction)
    multi_line_text("Confidence Score:", f"{confidence_score}%")
    multi_line_text("Disease Description:", description)
    multi_line_text("Precaution:", precaution)

    # Output PDF with a specific filename
    pdf_output = "disease_prediction_report.pdf"
    pdf.output(pdf_output)
    return pdf_output

# Dictionaries for descriptions and precautions
disease_descriptions = dict(zip(description_data['Disease'].str.lower(), description_data['Description']))
disease_precautions = dict(zip(precaution_data['Disease'].str.lower(), precaution_data[['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].apply(lambda x: ', '.join(x.fillna('')), axis=1)))

def get_disease_description(disease):
    return disease_descriptions.get(disease.lower(), "No description available.")

def get_disease_precaution(disease):
    return disease_precautions.get(disease.lower(), "No precaution available.")

In [ ]:
# Disease prediction
def predict_disease(symptoms_input):
    symptoms = symptoms_input.lower().split(",")
    symptoms = [symptom.strip() for symptom in symptoms]

    max_symptoms = 17
    symptoms = symptoms[:max_symptoms] + ["unknown"] * (max_symptoms - len(symptoms))

    symptoms_encoded = pd.DataFrame(0, index=[0], columns=X.columns)
    for i, symptom in enumerate(symptoms):
        if symptom in symptom_to_number:
            symptoms_encoded.iloc[0, i] = symptom_to_number[symptom]

    prediction = best_rf_model.predict(symptoms_encoded)[0]
    disease = label_encoder.inverse_transform([prediction])[0]

    confidence_score = get_confidence_score(symptoms_encoded)
    disease_description = get_disease_description(disease)
    disease_precaution = get_disease_precaution(disease)

    pdf_file = export_to_pdf(symptoms_input, disease, confidence_score, disease_description, disease_precaution)
    return disease, confidence_score, disease_description, disease_precaution, pdf_file

# Gradio Interface
def interface():
    with gr.Blocks() as interface:
        gr.Markdown("## Illness Intel: Disease Prediction System")

        symptom_input = gr.Textbox(label="Enter symptoms (separated by commas)", lines=2, placeholder="e.g., cough, fever, headache")
        prediction_output = gr.Textbox(label="Predicted Disease", interactive=False)
        confidence_output = gr.Textbox(label="Confidence Score (%)", interactive=False)
        description_output = gr.Textbox(label="Disease Description", interactive=False)
        precaution_output = gr.Textbox(label="Precaution", interactive=False)
        pdf_output = gr.File(label="Download Prediction Report (PDF)", interactive=False)

        def real_time_prediction(symptoms):
            if symptoms:
                prediction, confidence, description, precaution = predict_disease(symptoms)[:-1]
                return prediction, confidence, description, precaution
            return "", "", "", ""

        symptom_input.change(real_time_prediction, inputs=symptom_input, outputs=[prediction_output, confidence_output, description_output, precaution_output])

        def submit(symptoms):
            prediction, confidence, description, precaution, pdf_file = predict_disease(symptoms)
            return prediction, confidence, description, precaution, pdf_file

        gr.Button("Submit").click(submit, inputs=symptom_input, outputs=[prediction_output, confidence_output, description_output, precaution_output, pdf_output])

        def clear_fields():
            return "", "", "", "", None

        gr.Button("Clear").click(clear_fields, inputs=[], outputs=[symptom_input, prediction_output, confidence_output, description_output, precaution_output, pdf_output])

    return interface

# Launch the Gradio interface
interface().launch()
